# Práctica 2 - Parte III - Ejercicios SparkSQL

Ejercicios propuestos por el alumno


In [89]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("Ejemplo pySparkSQL") \
    .config("spark.sql.warehouse.dir", "file:///D:/tmp/spark-warehouse") \
    .getOrCreate()

sc = spark.sparkContext

25/12/07 18:20:30 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [90]:
%matplotlib inline 
import numpy as np
import matplotlib.pyplot as plt
from test_helper import Test
from pyspark.sql import functions as F
from pyspark.sql import Row

In [91]:
datosJSON = spark.read.json('datos/tweets2012/cache-0.json')

In [92]:
df = datosJSON.select('id', 'user.name', 'user.followers_count', 'text', 'retweet_count', 'place.country', 'entities.user_mentions', 'entities.hashtags', 'created_at')

In [93]:
Test.assertEquals(df.columns, ['id', 'name', 'followers_count', 'text', 'retweet_count',
                               'country', 'user_mentions', 'hashtags', 'created_at'],
                 'Columnas seleccionadas incorrectas')

1 test passed.


## Ejercicio 1. Usuarios con más retweets recibidos

En lugar de contar cuántos tweets ha escrito un usuario, calcula cuántos retweets ha conseguido en total sumando el retweet_count de todos sus mensajes. Muestra los 4 primeros.

In [94]:

top_retweeted_users = df.groupBy("name")\
                        .agg(F.sum("retweet_count").alias("total_retweets")) \
                        .sort(F.desc("total_retweets")) \
                        .take(4)

print(top_retweeted_users)

[Row(name='Travis Bennett  ', total_retweets=120480), Row(name='♥', total_retweets=98198), Row(name='Francisco Hernández', total_retweets=95288), Row(name='.', total_retweets=84259)]


In [95]:
Test.assertEquals([(r.name, r.total_retweets) for r in top_retweeted_users], [(u'Travis Bennett  ', 120480), (u'♥', 98198), (u'Francisco Hernández', 95288), (u'.', 84259)])

1 test passed.


## Ejercicio 2. Categorización de tweets por popularidad

Crea una nueva columna llamada popularity que clasifique los tweets según su número de retweets:

"Viral": más de 1000 retweets.

"Popular": entre 100 y 1000 retweets.

"Normal": menos de 100 retweets. Muestra un recuento de cuántos tweets hay en cada categoría.

In [96]:
df_popularity = df.withColumn("popularity", 
                              F.when(F.col("retweet_count") > 1000, "Viral")
                               .when((F.col("retweet_count") >= 100) & (F.col("retweet_count") <= 1000), "Popular")
                               .otherwise("Normal")).groupBy("popularity").count().collect()

In [97]:
Test.assertEquals([(r['popularity'], r['count']) for r in df_popularity], [(u'Viral', 29546), (u'Popular', 74502), (u'Normal', 895952)])

1 test passed.


## Ejercicio 3. Hashtags más largos

Al igual que hicimos con los hashtags más populares, esta vez queremos encontrar los hashtags más largos (con más caracteres). Usa explode, calcula la longitud del texto del hashtag y muestra los 10 hashtags únicos más largos.

In [98]:
long_hashtags = df.select(F.explode("hashtags").alias("tag")) \
                  .select(F.col("tag.text").alias("hashtag_text")) \
                  .withColumn("len", F.length("hashtag_text")) \
                  .distinct() \
                  .sort(F.desc("len")) \
                  .limit(10).collect()

long_hashtags


[Row(hashtag_text='Obamaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa', len=88),
 Row(hashtag_text='IfyouevendreamaboutsupportingMittyoubetterwakeupandapologize', len=60),
 Row(hashtag_text='QueFríoTanReSapoSetentaTripleDobleHijueputaGonorreaDeMierda', len=59),
 Row(hashtag_text='DontTellMeYoureGonnaBanBirthControlAndExpectAWomansVote', len=55),
 Row(hashtag_text='WhereWwreYouWhenTheWorldStoppedTurninThatSeptemberDay', len=53),
 Row(hashtag_text='twiteocomoridiculaqueselatirademuchoyesunadesechable', len=52),
 Row(hashtag_text='canwepleaserealizelacrosseisgrowingandstopignoringit', len=52),
 Row(hashtag_text='TweetWhatYouWereDoing11YearsAgoWhenTheTowersWereHit', len=51),
 Row(hashtag_text='whoreallygivesashitwhoitistheyallfuckshitupanyway', len=49),
 Row(hashtag_text='RebeldeSoVaiAcabarQuandoOUltimoFaPararDeRespirar', len=48)]

In [99]:
print([(r.hashtag_text, r.len) for r in long_hashtags])
long_hashtags = sorted(long_hashtags, key=lambda r: (-r['len'], r['hashtag_text']))
Test.assertEquals([(r['hashtag_text'], r['len']) for r in long_hashtags], [(u'Obamaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa', 88),
                                                                     (u'IfyouevendreamaboutsupportingMittyoubetterwakeupandapologize', 60), (u'QueFríoTanReSapoSetentaTripleDobleHijueputaGonorreaDeMierda', 59)
                                                                           , (u'DontTellMeYoureGonnaBanBirthControlAndExpectAWomansVote', 55), (u'WhereWwreYouWhenTheWorldStoppedTurninThatSeptemberDay', 53)
                                                                           ,(u'canwepleaserealizelacrosseisgrowingandstopignoringit', 52), (u'twiteocomoridiculaqueselatirademuchoyesunadesechable', 52)
                                                                            , (u'TweetWhatYouWereDoing11YearsAgoWhenTheTowersWereHit', 51), (u'whoreallygivesashitwhoitistheyallfuckshitupanyway', 49)
                                                                            , (u'RebeldeSoVaiAcabarQuandoOUltimoFaPararDeRespirar', 48)])

[('Obamaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa', 88), ('IfyouevendreamaboutsupportingMittyoubetterwakeupandapologize', 60), ('QueFríoTanReSapoSetentaTripleDobleHijueputaGonorreaDeMierda', 59), ('DontTellMeYoureGonnaBanBirthControlAndExpectAWomansVote', 55), ('WhereWwreYouWhenTheWorldStoppedTurninThatSeptemberDay', 53), ('twiteocomoridiculaqueselatirademuchoyesunadesechable', 52), ('canwepleaserealizelacrosseisgrowingandstopignoringit', 52), ('TweetWhatYouWereDoing11YearsAgoWhenTheTowersWereHit', 51), ('whoreallygivesashitwhoitistheyallfuckshitupanyway', 49), ('RebeldeSoVaiAcabarQuandoOUltimoFaPararDeRespirar', 48)]
1 test passed.


## Ejercicio 4. Usuarios "Influencers" de España

Filtra los usuarios que twittearon desde "Spain", que tengan más de 1000 seguidores y ordena el resultado por número de seguidores de forma descendente. Muestra el nombre y el conteo de seguidores, asegurándote de no mostrar duplicados (un mismo usuario puede tener varios tweets).

In [100]:
spain_influencers = df.filter((F.col("country") == "Spain") & (F.col("followers_count") > 1000)) \
                      .select("name", "followers_count") \
                      .dropDuplicates(["name"]) \
                      .sort(F.desc("followers_count")) \
                      .collect()

In [101]:
Test.assertEquals([(r.name, r.followers_count) for r in spain_influencers],
                  [('Joe Thorpe', 30546),
                   ('Carlos Bustillo', 15334),
                   ('Jose Blanco Oliver', 7479),
                   ('Andrés Puentes', 2133),
                   ('laura juanola', 1519),
                   ('joaquin nieto guerra', 1129),
                   ('FrancescLuna.com', 1087),
                   ("←Omar Habbab's Brain", 1085),
                   ('roaldcs', 1036)])

1 test passed.


In [102]:
mentions_impact = df.withColumn("num_mentions", F.size(F.col("user_mentions"))) \
                    .groupBy("num_mentions") \
                    .agg(F.avg("retweet_count").alias("avg_retweets"), F.count("id").alias("num_tweets")) \
                    .sort("num_mentions") \
                    .collect()

mentions_impact

[Row(num_mentions=0, avg_retweets=0.002432134152487833, num_tweets=386903),
 Row(num_mentions=1, avg_retweets=246.59663625663305, num_tweets=511454),
 Row(num_mentions=2, avg_retweets=28.655403865150326, num_tweets=78962),
 Row(num_mentions=3, avg_retweets=16.004812010778902, num_tweets=15586),
 Row(num_mentions=4, avg_retweets=2775.96688172043, num_tweets=4650),
 Row(num_mentions=5, avg_retweets=2.1650557620817845, num_tweets=1345),
 Row(num_mentions=6, avg_retweets=1.2046109510086456, num_tweets=694),
 Row(num_mentions=7, avg_retweets=1.5447154471544715, num_tweets=246),
 Row(num_mentions=8, avg_retweets=1.4479166666666667, num_tweets=96),
 Row(num_mentions=9, avg_retweets=1.2439024390243902, num_tweets=41),
 Row(num_mentions=10, avg_retweets=0.6, num_tweets=15),
 Row(num_mentions=11, avg_retweets=0.0, num_tweets=2),
 Row(num_mentions=12, avg_retweets=0.75, num_tweets=4),
 Row(num_mentions=14, avg_retweets=0.0, num_tweets=1),
 Row(num_mentions=15, avg_retweets=0.0, num_tweets=1)]

In [103]:
Test.assertEquals(
    [(r['num_mentions'], r['avg_retweets'], r['num_tweets']) for r in mentions_impact],
    [
    (0, 0.002432134152487833, 386903),
    (1, 246.59663625663305, 511454),
    (2, 28.655403865150326, 78962),
    (3, 16.004812010778902, 15586),
    (4, 2775.96688172043, 4650),
    (5, 2.1650557620817845, 1345),
    (6, 1.2046109510086456, 694),
    (7, 1.5447154471544715, 246),
    (8, 1.4479166666666667, 96),
    (9, 1.2439024390243902, 41),
    (10, 0.6, 15),
    (11, 0.0, 2),
    (12, 0.75, 4),
    (14, 0.0, 1),
    (15, 0.0, 1),
],
    'mentions_impact no coincide con lo esperado'
)

1 test passed.


## Ejercicios con RDD

Convertimos el DataFrame limpio a RDD

In [104]:
rdd_tweets = df.rdd

## Ejercicio 5. Conteo total de tweets por usuario
Calcula cuántos tweets ha publicado cada usuario.

In [105]:
tweets_por_usuario = rdd_tweets.map(lambda x: (x['name'], 1)) \
                               .reduceByKey(lambda a, b: a + b) \
                               .sortBy(lambda x: x[1], ascending=False) \
                               .take(10)

In [106]:
Test.assertEquals(tweets_por_usuario, [
    (u'The Action Group', 9762),
    (u'National Weather', 993),
    (u'TripleXDiamonds', 712),
    (u'BrillianceEngagement', 711),
    (u'BrilliantEngagement', 710),
    (u'EngagementDiamond', 694),
    (u'Bridal_Ringset', 690),
    (u'Celebrity Jewelry', 690),
    (u'FollowFriday Ranking', 637),
    (u'Psoriasis Care', 580),
])

1 test passed.


## Ejercicio 6. Desviación de seguidores (Min y Max por país)
Para cada país, queremos saber la diferencia entre el usuario con más seguidores y el que tiene menos (el rango).

In [107]:
rango_seguidores = rdd_tweets.filter(lambda x: x['country'] is not None) \
                             .map(lambda x: (x['country'], (x['followers_count'], x['followers_count']))) \
                             .reduceByKey(lambda a, b: (min(a[0], b[0]), max(a[1], b[1]))) \
                             .mapValues(lambda x: x[1] - x[0]) \
                             .sortBy(lambda x: x[1], ascending=False) \
                             .take(10)



In [108]:
print(rango_seguidores)
Test.assertEquals(
    rango_seguidores,
    [('United Kingdom', 618884), ('United States', 281241), ('Brasil', 93167), ('Turkey', 80691),
     ('Germany', 37263), ('France', 35442), ('Mexico', 32313), ('Spain', 30544),
     ('Greece', 29988), ('Chile', 28785)]
)

[('United Kingdom', 618884), ('United States', 281241), ('Brasil', 93167), ('Turkey', 80691), ('Germany', 37263), ('France', 35442), ('Mexico', 32313), ('Spain', 30544), ('Greece', 29988), ('Chile', 28785)]
1 test passed.


## Ejercicio 7. Promedio de seguidores por País (Patrón de Media)
Calcula el número medio de seguidores que tienen los usuarios por cada país

In [109]:
avg_followers = rdd_tweets.filter(lambda x: x['country'] is not None) \
                          .map(lambda x: (x['country'], (x['followers_count'], 1))) \
                          .reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1])) \
                          .map(lambda x: (x[0], x[1][0] / x[1][1])) \
                          .sortBy(lambda x: x[1], ascending=False) \
                          .take(10)



In [110]:
print(avg_followers)
Test.assertEquals(
    avg_followers,
    [('Armenia', 9358.0), ('Greece', 8204.5), ('Israel', 7857.0), ('United Kingdom', 4397.393782383419),
     ('United Arab Emirates', 3963.0), ('Japan', 2652.5416666666665), ('Ireland', 2482.5789473684213),
     ('Argentina', 2269.4736842105262), ('Egypt', 2068.0), ('Turkey', 1985.1346153846155)]
)

[('Armenia', 9358.0), ('Greece', 8204.5), ('Israel', 7857.0), ('United Kingdom', 4397.393782383419), ('United Arab Emirates', 3963.0), ('Japan', 2652.5416666666665), ('Ireland', 2482.5789473684213), ('Argentina', 2269.4736842105262), ('Egypt', 2068.0), ('Turkey', 1985.1346153846155)]
1 test passed.


## Ejercicio 8: Histograma de longitud de tweets (Buckets)
Cuenta cuántos tweets tienen una longitud entre 0-50, 51-100, y >100.

In [111]:
def categorizar_longitud(texto):
    l = len(texto)
    if l <= 50: return "Corto (0-50)"
    elif l <= 100: return "Medio (51-100)"
    else: return "Largo (>100)"

hist_longitud = rdd_tweets.map(lambda x: (categorizar_longitud(x['text']), 1)) \
                          .reduceByKey(lambda a, b: a + b) \
                          .collect()



In [112]:
print(hist_longitud)
Test.assertEquals(
    sorted(hist_longitud),
    sorted([('Medio (51-100)', 315813), ('Largo (>100)', 607461), ('Corto (0-50)', 76726)])
)

[('Medio (51-100)', 315813), ('Largo (>100)', 607461), ('Corto (0-50)', 76726)]
1 test passed.


## Ejercicio 9: Análisis de Popularidad por Nivel de Seguidores
Calcula la media del número de retweets para los usuarios que se clasifican en dos grupos según su followers_count:

"High Followers": Usuarios con más de 10,000 seguidores.

"Low Followers": Usuarios con 10,000 seguidores o menos.

La solución debe mostrar la media de retweets para cada uno de estos dos grupos.

In [113]:
def classify_followers(row):
    """Clasifica al usuario y devuelve (Clase, (Retweets, 1))"""
    followers = row['followers_count']
    retweets = row['retweet_count']
    
    if followers > 10000:
        key = "High Followers"
    else:
        key = "Low Followers"
        
    return (key, (retweets, 1))


avg_retweets_by_class = rdd_tweets.map(classify_followers) \
                                  .reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1])) \
                                  .map(lambda x: (x[0], x[1][0] / x[1][1])) \
                                  .collect()


In [114]:
print("Media de Retweets por Clase de Seguidores:")
print(avg_retweets_by_class)
Test.assertEquals(
    avg_retweets_by_class,
    [('Low Followers', 144.2561398249929), ('High Followers', 39.58442863769909)],
    'Media de retweets por clase no coincide'
)

Media de Retweets por Clase de Seguidores:
[('Low Followers', 144.2561398249929), ('High Followers', 39.58442863769909)]
1 test passed.


## Ejercicio 10: Creadores de Contenido y sus Menciones Únicas
Identifica cuántas personas diferentes mencionó cada usuario en todos sus tweets. Esto simula una unión entre dos conjuntos de datos creados a partir del mismo RDD.

RDD 1 (Autores): Conteo de cuántos tweets ha escrito cada usuario. (Clave: name).

RDD 2 (Menciones): Lista de todos los usuarios únicos que fueron mencionados por el autor. (Clave: name).

Join: Unir RDD 1 y RDD 2 por el name del autor.

In [115]:

rdd_conteo_tweets = rdd_tweets.map(lambda x: (x['name'], 1)) \
                              .reduceByKey(lambda a, b: a + b)

rdd_menciones_unicas = rdd_tweets.flatMap(
    lambda x: [(x['name'], m.screen_name) for m in x['user_mentions']]
) \
.groupByKey() \
.mapValues(lambda x: len(set(x)))

final_join_rdd = rdd_conteo_tweets.join(rdd_menciones_unicas) \
                                  .sortBy(lambda x: x[1][1], ascending=False)


resultado = final_join_rdd.take(10)

In [116]:
print("Top 10 Autores con más Menciones Únicas:")
print("Formato: (Autor, (Total Tweets, Menciones Únicas Realizadas))")
print(resultado)

Test.assertEquals(resultado, [
    ('The Action Group', (9762, 9760)),
    ('FollowFriday Ranking', (637, 614)),
    ('Fox News Alert', (518, 596)),
    ('Vicki Donovan', (276, 239)),
    ('John', (251, 212)),
    ('Government Robot', (160, 210)),
    ('Daniel J Sobieski', (214, 209)),
    ('Carolyn', (189, 197)),
    ('Max Emfinger', (52, 187)),
    ('.', (288, 181)),
], 'Los pares (tweets, menciones) no coinciden con lo esperado')

Top 10 Autores con más Menciones Únicas:
Formato: (Autor, (Total Tweets, Menciones Únicas Realizadas))
[('The Action Group', (9762, 9760)), ('FollowFriday Ranking', (637, 614)), ('Fox News Alert', (518, 596)), ('Vicki Donovan', (276, 239)), ('John', (251, 212)), ('Government Robot', (160, 210)), ('Daniel J Sobieski', (214, 209)), ('Carolyn', (189, 197)), ('Max Emfinger', (52, 187)), ('.', (288, 181))]
1 test passed.
